<br />

# 1-D Heat and Moisture Transfer Calculation  

<br />


## 1. Overview
本プログラムは前進型有限差分法を用いた多孔質材料内の熱・水分の移動計算を行うものである。  
空気や塩の移動や結氷といった現象への拡張も可能であるが、ここでは一般的な熱と水分（水蒸気・液水）の移動計算までの使用方法について述べる。本プログラムでは以下の計算が行える。  

### 本プログラムでできること

#### 計算内容（収支式）  
- 熱計算  
- 熱、水分計算（蒸気拡散支配領域）  
- 熱・水分計算（液水領域を含む）  

#### 流量計算方法  
- 熱（固体熱伝導）（水分の相変化熱）  
- 水蒸気（圧力基準　or 水分化学ポテンシャル基準）（絶対湿度基準も実装予定）  
- 液水（圧力基準　or 水分化学ポテンシャル基準）  

#### 境界条件  
- 第一種境界条件（温湿度固定）  
- 第二種境界条件（流量固定）  
- 第三種境界条件（セル間の流量計算）  


## 2. Dependency  
本プログラムの使用するにあたり必要な作業環境およびモジュールについて説明する。

### 2.1 使用言語
- python ver.3.6

### 2.2 作業環境  
- jupyternotebook  
    ※nbimporterの導入が必須  
    インターネット環境下でAnaconda Pronptから「 pip install nbimporter 」で導入可能

### 2.3 必要なライブラリ  
- numpy
- matplotlib  
- pandas
など

### 2.4 各モジュール（ファイル）の説明  

各モジュールの内容、および必要なファイル・ライブラリは以下に示すとおりである。

#### 各モジュールを統括するファイル
- 1D_heat_moisture_main.ipynb  
    内容；各モジュールを用いることでセルのセッティング、流量計算、収支計算までの一連の流れを示す。  
    必要ファイル；以下全てのモジュール  
  
  
#### Cellを構成するモジュール群
- gas_water.ipynb  
    内容；気体（水蒸気）の状態量・物質量を保持するモジュール  
    必要ファイル；moisture_conversion.ipynb  


- liquid_water.ipynb  
    内容；液体（純水）の状態量・物理量を保持するモジュール  
    必要ファイル；property_conversion.ipynb  


- solid_material.ipynb  
    内容；固体（材料骨格部）の状態量・物理量を保持するモジュール  
    必要ファイル；property_conversion.ipynb  


#### 流量計算に関するモジュール
- flux.ipynb  
    内容；熱・水分の流量計算式を管理するモジュール  
    必要ファイル；なし
   

#### 収支計算に関するモジュール  
- balance_equation.ipynb  
    内容；熱・水分の収支計算式を管理するモジュール  
    必要ファイル；なし


#### 材料の物性値に関するモジュール群   
- property_conversion.ipynb  
    内容；各物性値ファイルから必要な情報を取得し、流量計算に適した形に変換するモジュール  
    必要ファイル；vapour_pressure, 各種物性値ファイル


- van_genuchten.ipynb  
    内容；mualeam - vanGenuchtenの水分特性曲線から不飽和透水係数を算出するモデル  
    必要ファイル；なし  


#### 水分の状態量・物質量の変換に関するモジュール群  
⇒gas_waterモジュールに加えても良い気がする

- vapour_pressure.ipynb  
    内容；温度と水分化学ポテンシャルから水蒸気圧・飽和水蒸気圧・水蒸気圧の温度微分を求めるモジュール  
    必要ファイル；なし
    

- moisture_conversion.ipynb
    内容；水蒸気の水分状態（相対湿度・絶対湿度・水蒸気圧・水分化学ポテンシャル）を変換するモジュール  
    必要ファイル；vapour_pressure.ipynb 



## 3. 熱水分同時移動解析に関する基礎知識

本プログラムを使用するにあたり大切となる最も基本的な考え方を以下に示す。  
なお本プログラムは基本的には松本らが示した熱水分同時移動方程式を用い、前進型差分法によって壁体内の熱水分性状を解析するものである。その基本的な概念として多孔質材料内の構造や、熱や物質の移動原理について説明を行う。その説明に当たり計算上必要となる差分法の説明、および局所平衡仮定について簡単に示す。  

### 3.1 多孔質材料の構造  

　私たちの身の回りは数々の多孔質材料であふれている。多孔質材料とは無数の細孔を有する材料の総称であり、身近な例で言うと木材やコンクリート、土などが挙げられるだろう。細孔は材料内で連続的に繋がっている場合が多く、周囲の空気や水分などがその内部を自由に行き来することができるため、多孔質材料内の空間には材料実質部だけでなく空気や水蒸気、水や氷など様々な要素が含まれている。  
　そのため多孔質材料中の空隙部（細孔中）は単なる空間と考えても良いが、いわゆる大空間と大きく異なる点は空間の体積に対し固体実質部の面積が大きいことが挙げられる。固体表面では、固体内部とは異なり原子が周囲の物質と結合していないため自由エネルギーが大きく不安定であるため、固体表面に接近した分子やイオンなどの化学種を結合し自由エネルギーを少なくしようとする、”吸着”と呼ばれる現象が生じる。  
　多孔質材料中ではこの力および後述する毛管圧によって、空気中の水蒸気を固体表面に大量に吸着しようとする。そのため相対湿度が100%以下の環境下においても空気中の水分が材料表面に液体として保持されることがある。ここで平衡状態において相対湿度と液体として保持される水分量（含水率）の関係を平衡含水率・等温吸着線などと呼ぶ。なお多孔質材料中の水蒸気の量と液体として保持される量の関係は材料ごとに固有であるため、材料ごとに測定を行う必要がある。  
　なおこの平衡含水率関係は乾燥させた材料に水分を吸湿させていく時と、飽和させた材料から水分を放湿させていく場合とで異なる。これは吸放湿履歴現象と呼ばれ、今なお研究が盛んな分野ではあるが、本プログラムでは現状この現象について取り扱わないこととする。  
  ※材料は均質性・等方性を有しているとし、方向による違いがないものとする。

### 3.2 熱や物質の移動原理
#### 3.2.1 はじめに
　多孔質材料中が熱力学的に非平衡なとき、系全体が平衡となるよう熱や物質の移動や変化が生じる。  
　中でも建築材料中では以下の現象が頻繁に生じる。
- 材料中での熱の移動（固体熱伝導）
- 空気・水蒸気・液水などの移動
- 材料中における水分の状態変化（氷　―　水　―　水蒸気）  

　以下ではこれらの流れについて説明を行う。なお材料の種類によっては電気的な流れなども考える場合があるかもしれないがここでは割愛する。  

#### 3.2.2 熱の移動
　一般に材料実質部に温度差がある場合、物質の移動を伴わず高温側から低温側に熱が移動する。この物理現象を説明する有名な法則としてフーリエの熱伝導法則が存在する。

Fourierの法則　：　$q = -\lambda \nabla T$  

$q$：熱流[W/m2s]  
$\lambda$：熱伝導率[W/msK]  
$T$：絶対温度[K]  

　本プログラムでは材料間の熱の移動はフーリエの法則に基づいているとし、熱流は伝熱現象によって生じるとする。
 
　一方で材料表面と空気間での熱の移動は主に2種類に大別される。1つは材料表面と空気が直接熱を交換し、それが空気間で共有されることによって伝わる対流熱伝達。もう一つは熱放射による熱の交換であり、材料と空気がそれぞれ温度に応じた電磁波を発し、互いが電磁波を吸収することで熱のやり取りを行う放射熱伝達である。  
 
熱伝達　：　$q = -\alpha \Delta T$  

$q$：熱流[W/m2s]  
$\alpha$：熱伝達率[W/m2sK]  
$T$：絶対温度[K]  

#### 3.2.3 物質の相変化
　3.1節で示すように多孔質材料中では水分が水蒸気と液水の2つの状態で保持される。そのため材料中では水分が蒸発する際に周囲の熱を吸収したり、水蒸気が凝縮し液水になる際に熱を放出したりする。この時に吸熱・放熱される熱量のことを一般に潜熱と呼ぶ。相変化の量については様々な考え方が存在するが、ここでは後述の局所平衡の考えに則り、微小空間における水蒸気と液水の水分状態（水分の化学ポテンシャル）が常に平衡していると考えることでその量を求める。  
　基本的な計算方法は以下に示すとおりですある。まず材料中の微小時間当たりの水蒸気の収支式は以下のように表される。  

水蒸気の収支式　：　$\frac{\partial(\phi_0-\psi)\rho_v}{\partial t} = - \frac{\partial J_{1w}}{\partial x} - W$  

　$\phi$：材料の空隙率、$\psi$：含水率、$\rho_v$：水蒸気の密度、$t$：時間、$J_{1w}$：水蒸気の移動量、$W$：水蒸気から液水への相変化量である。ここで、微小時間における水蒸気量の変化量は非常に小さいと考えると、$\frac{\partial(\phi_0-\psi)\rho_v}{\partial t} \equiv 0 $となる。よって水分の相変化量を、  

水蒸気から液水への相変化量　：　$ W = - \frac{\partial J_{1w}}{\partial x} $  

と考えた。

#### 3.2.3 物質の移動
　多孔質材料中は空気や水蒸気、液水など様々な流体が存在し、熱力学的非平衡に基づき移動している。これら物質の移動原理に関しては様々な考え方があるが、大きくはその物質の濃度差やポテンシャル差によって移動が生じる拡散と、他の物質の移動に伴って流れる移流に大別されるだろう。これらの移動原理のうち空気中や溶液中の物質の拡散移動に関してはFickの拡散法則が有名であろう。

Fickの法則　：　$J = -D \nabla c_s$  
$J$：物質流れ[kg/m2s]  
$D$：拡散係数[m2/s]  
$c_s$：物質の濃度[kg/m3]  

　一方で、土壌など物質内を流れる液には濃度差が存在しないが、動水勾配によって液水が流れると考えることで以下のダルシー則が用いられる場合が多い。  

ダルシー則　：　$v = k i $  
$v$：見かけの浸透流速[m/s]  
$k$：透水係数[m/s]  
$i$：動水勾配

　ただし、本プログラムで用いる松本らの熱水分同時移動方程式は、熱力学的非平衡の観点から（基本は熱力学第二法則従うと考える）物質は移動する、というのを基本としており、式の出発点はこれらの法則とはやや異なる。（Fickの法則やダルシー則に矛盾はしないが）

#### - 水分移動の原理について
　本プログラムでは水分は概ね上述のFickの法則やダルシー則に従い流れが生じる。気相空間では一般に圧力差が生じないことから拡散現象により水蒸気が流れ、液相空間では毛管圧差によってダルシー則に従った水分の流れが生じる。

#### 材料中における水分の移動しやすさ
　また多孔質材料中における水蒸気や液水の移動量は物質の量だけでなくその連続性によっても変化する。例えば材料中において液水が全くない状態と、細孔が液水で埋まっている状態を比べた場合、前者では水蒸気が細孔中を簡単に通過できるのに対し、後者では液水がその移動を妨げるため水蒸気の移動量が減少する。同様に材料中に液水がほとんどない場合と材料が液水でほぼ飽和している状態とでは後者の方が圧倒的に液水の移動速度は上昇する。このように、材料中の水分の移動は水分の飽和状態によっても左右されるものである。
 

### 3.3 局所平衡
　多孔質材料中における局所平衡とは、材料中のある部分の熱力学的系が熱的、力学的、化学的に平衡であることを指す。そのためこの部分では熱や物質の移動が生じず平衡状態に達しているとみなすことができる。  
　プログラム中ではある微小空間における固相・液相・気相全ての温度が一致しており、水分など各種物質の化学ポテンシャルが平衡していることを指す。実際の材料中において局所平衡が成り立っているかどうかについては議論の余地が残るが、一般に十分に小さい系では材料実質部および液水、水蒸気の温度は平衡しており、液水と水蒸気の化学ポテンシャルが平衡しているという仮定を置くことで非常に計算が簡単となるためこの仮定を置く場合が多い。  


### 3.4 有限差分法
　有限差分法（単に差分法と呼ばれる場合もある）とは、「微分方程式を解くために微分を有限差分近似（差分商）で置き換えて得られる差分方程式で近似するという離散化手法を用いる数値解法である。」（wikipediaより）一般に壁体内部の熱や水分の移動量を計算するには壁をある微小長さごとに分割し（差分し）、その微小領域ごとの熱や水分の移動を計算することとなる。差分法と異なる計算方法として有限体積法・有限要素法などがある。  
　差分法のメリットに計算が単純化しやすく分かりやすいことが挙げられるが、一方で離散化による誤差が大きく、精度検証のためフォンノイマンの安定性解析などを行う必要がある。また差分法の計算方法には陽解法と陰解法、クランク・ニコルソン法などが存在するが、本プログラムでは陽解法による熱水分移動解析の方法を示した。

## 4. Description

本プログラムを使用するにあたり大切となる最も基本的な考え方を以下に示す。  
なお本プログラムは基本的には松本らが示した熱水分同時移動方程式を用い、前進型差分法によって壁体内の熱水分性状を解析するものである。その基本的な概念として多孔質材料内の構造や、熱や物質の移動原理について説明を行う。その説明に当たり計算上必要となる差分法の説明、および局所平衡仮定について簡単に示す。  
なお各種計算に必要な方程式についての説明はそれぞれのモジュールに詳細を記載してあるためここでは割愛する。  


<img src="picture/readme_no1.jpg" width= 60%>  

### 4.1 セルとは  
　本プログラムにおけるセルとは上述の差分刻みによって分割された壁体の最小単位を示す。多孔質材料内部の構造は上述した通りであるが、改めてその構造について記す。  
- セルとは複数の要素によって構成され、その要素数に制約はない。
- 主な構成要素は固体（材料骨格部・氷・塩結晶）、液体（水・溶液）、気体（空気・水蒸気）である。
- それぞれの構成要素は異なる物理量・状態量を保持している。
- ただし一般にセル内では局所平衡が成り立ち、3相の温度や水分の状態は平衡しているものと仮定する。 
- 材料の物性値および各種移動係数は
水分伝導率などの各種移動係数は、固体と液体、気体全ての物理量に影響を受けるため、各要素が保持するというよりはセルが有する情報とする。

### 4.2 流量計算  
- 流れとは、隣り合う2つのセルが熱力学的非平衡状態になった際に生じるものである。  
- 基本的に熱や物質の流れは隣接するセルのうち、同じ要素の間で生じるものである。（隣接するセル間では液水と材料の間で熱伝導は生じない）
- 


- セルの質点の置き方について
    セルの分割方法、質点の分割の仕方についてはいくつか方法が考えられる。
    - 質点を全てセルの中心に置く方法（収支計算が簡単）　⇔　
    - 質点を全てセルの端に置く方法（流量計算が簡単）　⇔　異なる材料の水分収支を解く際に平衡含水率をどうするのかが難しい
    - 境界面のセルのみ質点を端に置き、他はすべてセルの中央に置く方法（質点間の距離計算が複雑）　⇒　計算結果はきれいだがプログラムが膨大に
    - 質点を全てセルの中心に置き、境界面のセルのみ体積を半分にする方法（体積の計算が複雑）

### 4.3 境界条件  
壁の境界（端のセル）では境界内部で成り立つ方程式だけでは解を決定できないため、補助的な拘束条件を課す必要がある。  
境界条件は主に三種類存在し以下のように分類できる。

- 第一種境界条件（境界温湿度固定）  
- 第二種境界条件（流量固定）  
- 第三種境界条件（境界に隣接するからの流量計算）   

第三種境界条件を用いて空気と壁表面の流量計算を行う場合、セルクラスと同様に空気もクラス化することで流量の計算を行うことが好ましい。
この際セルクラスと同様、空気のクラスには空気の物性値および対流伝達率（または総合伝達率など）を保持する。

### 4.4 収支計算

### 4.5 計算時間に関するモジュール

### 4.6 データの入出力

### 3.7 エラーチェック  

フォン・ノイマンの安定性解析などが有用であろう

### 3.8 計算のフローチャート   
本プログラムにおける基本的な計算手順は以下のようになる。  

<img src="caluculationMethod.jpg" width= 60%>  


## 4. Demo

## 5. License

## 6. Authors

## 7. References